<a href="https://colab.research.google.com/github/GitFazz/POS-Tagging-kapjhap/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT all modules

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.utils
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer

Preparing dataset

In [ ]:

fp = open("base_dataset.txt","r")

X = []
Y = []

while True:
    line = fp.readline()
    if not line: break

    words = line.split(" ")

    X_sentence = []
    Y_sentence = []

    for w in words:
        com = w.split("\\")

        one = com[0]
        two = com[1].split(".",2)[0]

        if len(two) < 1 : continue

        # filtering dataset
        if (not two.startswith("PU")) and (not two.startswith("R")) and (not two.startswith("?")) :
          
          X_sentence.append(one) # add the word

          if (two.startswith("PP")) : 
            Y_sentence.append("PP")
          else :
            Y_sentence.append(two[0]) #otherwise take only the fist letter
        

    X.append(X_sentence)
    Y.append(Y_sentence)


  

fp.close()

            
    


Testing dataset

In [ ]:
# let’s look at first data point
# this is one data point that will be fed to the RNN
print('  sample X: ', X[0], '\n')
print('sample Y: ', Y[0], '\n')

  sample X:  ['রপ্তানি', 'দ্রব্য', 'তাজা', 'ও', 'শুকনা', 'ফল', 'আফিম', 'পশুচর্ম', 'ও', 'পশম', 'এবং', 'কার্পেট'] 

sample Y:  ['J', 'N', 'J', 'C', 'J', 'N', 'N', 'N', 'C', 'N', 'C', 'N'] 



Printing labels and size

In [ ]:
temp = []
for i in Y :
  for j in i :
    if j not in temp:
        temp.append(j)
print(temp)
print('size : ',len(temp))

['J', 'N', 'C', 'V', 'P', 'D', 'PP', 'A', 'L']
size :  9


Tokenizer

In [ ]:
from keras.preprocessing.text import Tokenizer

# encode X
word_tokenizer = Tokenizer()              # instantiate tokeniser
word_tokenizer.fit_on_texts(X)            # fit tokeniser on data
# use the tokeniser to encode input sequence
X_encoded = word_tokenizer.texts_to_sequences(X)  
# encode Y
tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(Y)
Y_encoded = tag_tokenizer.texts_to_sequences(Y)
# look at first encoded data point
print("** Raw data point **", "\n", "-"*100, "\n")
print('X: ', X[0], '\n')
print('Y: ', Y[0], '\n')
print()
print("** Encoded data point **", "\n", "-"*100, "\n")
print('X: ', X_encoded[0], '\n')
print('Y: ', Y_encoded[0], '\n')


** Raw data point ** 
 ---------------------------------------------------------------------------------------------------- 

X:  ['রপ্তানি', 'দ্রব্য', 'তাজা', 'ও', 'শুকনা', 'ফল', 'আফিম', 'পশুচর্ম', 'ও', 'পশম', 'এবং', 'কার্পেট'] 

Y:  ['J', 'N', 'J', 'C', 'J', 'N', 'N', 'N', 'C', 'N', 'C', 'N'] 


** Encoded data point ** 
 ---------------------------------------------------------------------------------------------------- 

X:  [1215, 1216, 1643, 2, 3975, 502, 3976, 3977, 2, 3978, 5, 3979] 

Y:  [3, 1, 3, 5, 3, 1, 1, 1, 5, 1, 5, 1] 



Padding

In [ ]:
# Pad each sequence to MAX_SEQ_LENGTH using KERAS’ pad_sequences() function. 
# Sentences longer than MAX_SEQ_LENGTH are truncated.
# Sentences shorter than MAX_SEQ_LENGTH are padded with zeroes.
# Truncation and padding can either be ‘pre’ or ‘post’. 
# For padding we are using ‘pre’ padding type, that is, add zeroes on the left side.
# For truncation, we are using ‘post’, that is, truncate a sentence from right side.
# sequences greater than 100 in length will be truncated

from keras.preprocessing.sequence import pad_sequences 

MAX_SEQ_LENGTH = 90
X_padded = pad_sequences(X_encoded, maxlen=MAX_SEQ_LENGTH, padding='pre', truncating='post')
Y_padded = pad_sequences(Y_encoded, maxlen=MAX_SEQ_LENGTH, padding='pre', truncating='post')
# print the first sequence
print(X_padded[0], "\n"*3)
print(Y_padded[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0 1215 1216 1643    2 3975  502
 3976 3977    2 3978    5 3979] 



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 3 1 3 5 3 1 1 1 5 1 5 1]


Longest sentence

In [ ]:
# check length of longest sentence
lengths = [len(seq) for seq in X_encoded]
print("Length of longest sentence: {}".format(max(lengths)))

Length of longest sentence: 96


Update X and Y

In [ ]:
X, Y = X_padded, Y_padded

Embedding

In [ ]:

# assign word vectors from word2vec model
# each word in word2vec model is represented using a 300 dimensional vector
EMBEDDING_SIZE  = 300  
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1
NUM_CLASSES = 9
# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))
# create a word to index dictionary mapping
word2id = word_tokenizer.word_index
# copy vectors from word2vec model to the words present in corpus


Use one-hot encoding for output sequences (Y)

In [ ]:
# use Keras' to_categorical function to one-hot encode Y
Y = to_categorical(Y)
print(Y.shape[2])

10


Splitting test and train

In [ ]:
TEST_SIZE = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=4)

Vanilla RNN

Let’s start with the first experiment: a vanilla RNN with arbitrarily initialized, untrainable embedding. For this RNN we won’t use the pre-trained word embeddings. We’ll use randomly initialized embeddings. Moreover, we won’t update the embeddings weights.

In [ ]:
# create architecture
rnn_model = keras.Sequential()
# create embedding layer — usually the first layer in text problems
# vocabulary size — number of unique words in data
rnn_model.add(layers.Embedding(input_dim = VOCABULARY_SIZE, 
# length of vector with which each word is represented
 output_dim = EMBEDDING_SIZE, 
# length of input sequence
 input_length = MAX_SEQ_LENGTH, 
# False — don’t update the embeddings
 trainable = True 
))
# add an RNN layer which contains 64 RNN cells
# True — return whole sequence; False — return single output of the end of the sequence
rnn_model.add(layers.SimpleRNN(64, 
 return_sequences=True
))
# add time distributed (output at each sequence) layer

rnn_model.add(layers.TimeDistributed(layers.Dense(NUM_CLASSES+1, activation='softmax')))
#compile model
rnn_model.compile(loss      =  'categorical_crossentropy',
                  optimizer =  'adam',
                  metrics   =  ['acc']) 
# check summary of the model
rnn_model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 90, 300)           3685500   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 90, 64)            23360     
_________________________________________________________________
time_distributed (TimeDistri (None, 90, 10)            650       
Total params: 3,709,510
Trainable params: 3,709,510
Non-trainable params: 0
_________________________________________________________________


Vanila RNN fit model

In [ ]:
rnn_training = rnn_model.fit(X_train,Y_train,batch_size=128,epochs=10, validation_data=(X_test, Y_test))

Epoch 1/10
16/16 [==============================] - 4s 180ms/step - loss: 1.6663 - acc: 0.6734 - val_loss: 0.5677 - val_acc: 0.8695
Epoch 2/10
16/16 [==============================] - 2s 142ms/step - loss: 0.5402 - acc: 0.8697 - val_loss: 0.4069 - val_acc: 0.8893
Epoch 3/10
16/16 [==============================] - 2s 145ms/step - loss: 0.3761 - acc: 0.9036 - val_loss: 0.3306 - val_acc: 0.8900
Epoch 4/10
16/16 [==============================] - 2s 140ms/step - loss: 0.2959 - acc: 0.9089 - val_loss: 0.2730 - val_acc: 0.9203
Epoch 5/10
16/16 [==============================] - 2s 141ms/step - loss: 0.2383 - acc: 0.9361 - val_loss: 0.2313 - val_acc: 0.9318
Epoch 6/10
16/16 [==============================] - 2s 141ms/step - loss: 0.1952 - acc: 0.9436 - val_loss: 0.2007 - val_acc: 0.9392
Epoch 7/10
16/16 [==============================] - 2s 141ms/step - loss: 0.1646 - acc: 0.9509 - val_loss: 0.1779 - val_acc: 0.9481
Epoch 8/10
16/16 [==============================] - 2s 144ms/step - loss: 0.

Vanila RNN Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

pred = rnn_model.predict(X_test)
predict = np.array([np.argmax(y, axis=1) for y in pred])
actual = np.array([np.argmax(y, axis=1) for y in Y_test])

predict = predict.flatten()
actual = actual.flatten()

true_pos = 0
false_pos = 0
true_neg = 0
false_neg = 0

for j in range(1, NUM_CLASSES+1):
  true_pos += sum(1 for i in range(len(predict)) if (predict[i]==j) & (actual[i]==j))
  false_pos += sum(1 for i in range(len(predict)) if (predict[i]==j) & (actual[i]!=j))
  true_neg += sum(1 for i in range(len(predict)) if (predict[i]!=j) & (actual[i]!=j))
  false_neg += sum(1 for i in range(len(predict)) if (predict[i]!=j) & (actual[i]==j))

accuracy = (true_pos+true_neg)/(true_neg+true_pos+false_neg+false_pos)
precision = true_pos/(true_pos+false_pos)
recall = true_pos/(true_pos+false_neg)
f_score = 2.0*precision*recall/(precision+recall)
print("Accuracy: %.3f" % (accuracy))
print("F1 score: %.3f" % (f_score))

Accuracy: 0.993
F1 score: 0.758


LSTM

In [ ]:
# create architecture
lstm_model = Sequential()
# vocabulary size — number of unique words in data
# length of vector with which each word is represented
lstm_model.add(Embedding(input_dim = VOCABULARY_SIZE,
output_dim = EMBEDDING_SIZE,
# length of input sequence
input_length = MAX_SEQ_LENGTH,
# True — update embeddings_weight matrix
trainable = True
))
# add an LSTM layer which contains 64 LSTM cells
# True — return whole sequence; False — return single output of the end of the sequence
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(TimeDistributed(Dense(NUM_CLASSES+1,
activation='softmax')))
#compile model
lstm_model.compile(loss = 'categorical_crossentropy',
                          optimizer = 'adam',
                          metrics = ['acc'])
# check summary of the model
lstm_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 80, 300)           3685500   
_________________________________________________________________
lstm_1 (LSTM)                (None, 80, 64)            93440     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 80, 10)            650       
Total params: 3,779,590
Trainable params: 3,779,590
Non-trainable params: 0
_________________________________________________________________


LSTM train

In [ ]:
lstm_training = lstm_model.fit(X_train,Y_train,batch_size=128,epochs=10, validation_data=(X_test, Y_test))

Epoch 1/10
16/16 [==============================] - 7s 349ms/step - loss: 1.8420 - acc: 0.6898 - val_loss: 0.6270 - val_acc: 0.8695
Epoch 2/10
16/16 [==============================] - 5s 323ms/step - loss: 0.5871 - acc: 0.8694 - val_loss: 0.4155 - val_acc: 0.8794
Epoch 3/10
16/16 [==============================] - 5s 317ms/step - loss: 0.3898 - acc: 0.8924 - val_loss: 0.3314 - val_acc: 0.9043
Epoch 4/10
16/16 [==============================] - 5s 318ms/step - loss: 0.3120 - acc: 0.9111 - val_loss: 0.2807 - val_acc: 0.9200
Epoch 5/10
16/16 [==============================] - 5s 318ms/step - loss: 0.2668 - acc: 0.9229 - val_loss: 0.2524 - val_acc: 0.9241
Epoch 6/10
16/16 [==============================] - 5s 319ms/step - loss: 0.2335 - acc: 0.9286 - val_loss: 0.2348 - val_acc: 0.9254
Epoch 7/10
16/16 [==============================] - 5s 322ms/step - loss: 0.2221 - acc: 0.9273 - val_loss: 0.2213 - val_acc: 0.9269
Epoch 8/10
16/16 [==============================] - 5s 317ms/step - loss: 0.

LSTM Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

pred = lstm_model.predict(X_test)
predict = np.array([np.argmax(y, axis=1) for y in pred])
actual = np.array([np.argmax(y, axis=1) for y in Y_test])

predict = predict.flatten()
actual = actual.flatten()

true_pos = 0
false_pos = 0
true_neg = 0
false_neg = 0

for j in range(1, NUM_CLASSES+1):
  true_pos += sum(1 for i in range(len(predict)) if (predict[i]==j) & (actual[i]==j))
  false_pos += sum(1 for i in range(len(predict)) if (predict[i]==j) & (actual[i]!=j))
  true_neg += sum(1 for i in range(len(predict)) if (predict[i]!=j) & (actual[i]!=j))
  false_neg += sum(1 for i in range(len(predict)) if (predict[i]!=j) & (actual[i]==j))

accuracy = (true_pos+true_neg)/(true_neg+true_pos+false_neg+false_pos)
precision = true_pos/(true_pos+false_pos)
recall = true_pos/(true_pos+false_neg)
f_score = 2.0*precision*recall/(precision+recall)
print("Accuracy: %.3f" % (accuracy))
print("F1 score: %.3f" % (f_score))

Accuracy: 0.986
F1 score: 0.521


In [ ]:
GRU

In [ ]:
# create architecture
gru_model = Sequential()
# vocabulary size — number of unique words in data
# length of vector with which each word is represented
gru_model.add(Embedding(input_dim = VOCABULARY_SIZE,
output_dim = EMBEDDING_SIZE,
# length of input sequence
input_length = MAX_SEQ_LENGTH,

# True — update embeddings_weight matrix
trainable = True
))
# add an LSTM layer which contains 64 LSTM cells
# True — return whole sequence; False — return single output of the end of the sequence

gru_model.add(GRU(64, return_sequences=True))
gru_model.add(TimeDistributed(Dense(NUM_CLASSES+1,
activation='softmax')))
#compile model
gru_model.compile(loss = 'categorical_crossentropy', 
                  optimizer = 'adam',
                  metrics = ['acc'])

# check summary of the model
gru_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 90, 300)           3685500   
_________________________________________________________________
gru (GRU)                    (None, 90, 64)            70272     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 90, 10)            650       
Total params: 3,756,422
Trainable params: 3,756,422
Non-trainable params: 0
_________________________________________________________________


GRU Training

In [ ]:
gru_training = gru_model.fit(X_train,Y_train,batch_size=128,epochs=10, validation_data=(X_test, Y_test))

Epoch 1/10
16/16 [==============================] - 6s 302ms/step - loss: 1.9096 - acc: 0.7102 - val_loss: 0.5612 - val_acc: 0.8696
Epoch 2/10
16/16 [==============================] - 4s 272ms/step - loss: 0.4575 - acc: 0.8699 - val_loss: 0.3555 - val_acc: 0.8898
Epoch 3/10
16/16 [==============================] - 4s 275ms/step - loss: 0.3173 - acc: 0.9079 - val_loss: 0.2901 - val_acc: 0.9193
Epoch 4/10
16/16 [==============================] - 4s 272ms/step - loss: 0.2749 - acc: 0.9320 - val_loss: 0.2604 - val_acc: 0.9300
Epoch 5/10
16/16 [==============================] - 4s 271ms/step - loss: 0.2398 - acc: 0.9387 - val_loss: 0.2319 - val_acc: 0.9332
Epoch 6/10
16/16 [==============================] - 4s 273ms/step - loss: 0.2079 - acc: 0.9370 - val_loss: 0.2037 - val_acc: 0.9361
Epoch 7/10
16/16 [==============================] - 4s 277ms/step - loss: 0.1715 - acc: 0.9436 - val_loss: 0.1761 - val_acc: 0.9469
Epoch 8/10
16/16 [==============================] - 4s 277ms/step - loss: 0.

GRU Test

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

pred = gru_model.predict(X_test)
predict = np.array([np.argmax(y, axis=1) for y in pred])
actual = np.array([np.argmax(y, axis=1) for y in Y_test])

predict = predict.flatten()
actual = actual.flatten()

true_pos = 0
false_pos = 0
true_neg = 0
false_neg = 0

for j in range(1, NUM_CLASSES+1):
  true_pos += sum(1 for i in range(len(predict)) if (predict[i]==j) & (actual[i]==j))
  false_pos += sum(1 for i in range(len(predict)) if (predict[i]==j) & (actual[i]!=j))
  true_neg += sum(1 for i in range(len(predict)) if (predict[i]!=j) & (actual[i]!=j))
  false_neg += sum(1 for i in range(len(predict)) if (predict[i]!=j) & (actual[i]==j))

accuracy = (true_pos+true_neg)/(true_neg+true_pos+false_neg+false_pos)
precision = true_pos/(true_pos+false_pos)
recall = true_pos/(true_pos+false_neg)
f_score = 2.0*precision*recall/(precision+recall)
print("Accuracy: %.3f" % (accuracy))
print("F1 score: %.3f" % (f_score))

Accuracy: 0.994
F1 score: 0.797


Bidirectional LSTM

In [ ]:
# create architecture
bidirect_model = Sequential()
bidirect_model.add(Embedding(input_dim = VOCABULARY_SIZE,
output_dim = EMBEDDING_SIZE,
input_length = MAX_SEQ_LENGTH,
trainable = True
))
bidirect_model.add(Bidirectional(LSTM(64, return_sequences=True)))
bidirect_model.add(TimeDistributed(Dense(NUM_CLASSES+1,
activation='softmax')))
#compile model
bidirect_model.compile(loss='categorical_crossentropy',
                       optimizer='adam',
                       metrics=['acc'])
# check summary of model
bidirect_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 90, 300)           3685500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 90, 128)           186880    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 90, 10)            1290      
Total params: 3,873,670
Trainable params: 3,873,670
Non-trainable params: 0
_________________________________________________________________


Bidirectional Training

In [ ]:
bidirect_training = bidirect_model.fit(X_train,Y_train,batch_size=128,epochs=20, validation_data=(X_test, Y_test))

Epoch 1/20
16/16 [==============================] - 14s 679ms/step - loss: 1.6400 - acc: 0.6982 - val_loss: 0.4153 - val_acc: 0.8695
Epoch 2/20
16/16 [==============================] - 10s 615ms/step - loss: 0.3606 - acc: 0.8772 - val_loss: 0.2793 - val_acc: 0.9206
Epoch 3/20
16/16 [==============================] - 10s 616ms/step - loss: 0.2786 - acc: 0.9228 - val_loss: 0.2512 - val_acc: 0.9282
Epoch 4/20
16/16 [==============================] - 10s 619ms/step - loss: 0.2487 - acc: 0.9276 - val_loss: 0.2350 - val_acc: 0.9296
Epoch 5/20
16/16 [==============================] - 10s 616ms/step - loss: 0.2336 - acc: 0.9292 - val_loss: 0.2239 - val_acc: 0.9303
Epoch 6/20
16/16 [==============================] - 10s 618ms/step - loss: 0.2238 - acc: 0.9290 - val_loss: 0.2146 - val_acc: 0.9301
Epoch 7/20
16/16 [==============================] - 10s 620ms/step - loss: 0.2086 - acc: 0.9309 - val_loss: 0.2052 - val_acc: 0.9307
Epoch 8/20
16/16 [==============================] - 10s 617ms/step - 

Bidirectional testing

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

pred = bidirect_model.predict(X_test)
predict = np.array([np.argmax(y, axis=1) for y in pred])
actual = np.array([np.argmax(y, axis=1) for y in Y_test])

predict = predict.flatten()
actual = actual.flatten()

true_pos = 0
false_pos = 0
true_neg = 0
false_neg = 0

for j in range(1, NUM_CLASSES+1):
  true_pos += sum(1 for i in range(len(predict)) if (predict[i]==j) & (actual[i]==j))
  false_pos += sum(1 for i in range(len(predict)) if (predict[i]==j) & (actual[i]!=j))
  true_neg += sum(1 for i in range(len(predict)) if (predict[i]!=j) & (actual[i]!=j))
  false_neg += sum(1 for i in range(len(predict)) if (predict[i]!=j) & (actual[i]==j))

accuracy = (true_pos+true_neg)/(true_neg+true_pos+false_neg+false_pos)
precision = true_pos/(true_pos+false_pos)
recall = true_pos/(true_pos+false_neg)
f_score = 2.0*precision*recall/(precision+recall)
print("Accuracy: %.3f" % (accuracy))
print("F1 score: %.3f" % (f_score))

Accuracy: 0.996
F1 score: 0.859
